In [17]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
from tqdm import tqdm

In [18]:
class Environment():
    
    def __init__(self):
    # 보드는 0으로 초기화된 16개의 배열로 준비
    # 게임종료 : done = True
        self.board_a = np.zeros(16)
        self.done = False
        self.reward = 0
        self.winner = 0
        self.print = False

    def move(self, p1, p2, player):
    # 각 플레이어가 선택한 행동을 표시 하고 게임 상태(진행 또는 종료)를 판단
    # p1 = 1, p2 = -1로 정의
    # 각 플레이어는 행동을 선택하는 select_action 메서드를 가짐
        if player == 1:
            pos = p1.select_action(env, player)
        else:
            pos = p2.select_action(env, player)
        
        # 보드에 플레이어의 선택을 표시
        self.board_a[pos] = player
        if self.print:
            print(player)
            self.print_board()
        # 게임이 종료상태인지 아닌지를 판단
        self.end_check(player)
        
        return  self.reward, self.done
 
    # 현재 보드 상태에서 가능한 행동(둘 수 있는 장소)을 탐색하고 리스트로 반환
    def get_action(self):
        observation = []
        for i in range(16):
            if self.board_a[i] == 0:
                observation.append(i)
        return observation
    
    # 게임이 종료(승패 또는 비김)됐는지 판단
    def end_check(self,player):
        # 0 1 2 3
        # 4 5 6 7
        # 8 9 10 11
        # 12 13 14 15
        # 승패 조건은 가로, 세로, 대각선 이 -1 이나 1 로 동일할 때 
        end_condition = ((0,1,2,3),(4,5,6,7),(8,9,10,11),(12,13,14,15),(0,5,10,15),(3,6,9,12), (0,4,8,12), (1,5,9,13), (2,6,10,14), (3,7,11,15))
        for line in end_condition:
            if self.board_a[line[0]] == self.board_a[line[1]] \
                and self.board_a[line[1]] == self.board_a[line[2]] \
                and self.board_a[line[2]] == self.board_a[line[3]] \
                and self.board_a[line[0]] != 0:
                # 종료됐다면 누가 이겼는지 표시
                self.done = True
                self.reward = player
                return
        # 비긴 상태는 더는 보드에 빈 공간이 없을때
        observation = self.get_action()
        if (len(observation)) == 0:
            self.done = True
            self.reward = 0            
        return
        
    # 현재 보드의 상태를 표시 p1 = O, p2 = X    
    def print_board(self):
        print("+----+----+----+----+")
        for i in range(4):
            for j in range(4):
                if self.board_a[4*i+j] == 1:
                    print("|  O",end=" ")
                elif self.board_a[4*i+j] == -1:
                    print("|  X",end=" ")
                else:
                    print("|   ",end=" ")
            print("|")
            print("+----+----+----+----+")

In [19]:
class Human_player():
    
    def __init__(self):
        self.name = "Human player"
        
    def select_action(self, env, player):
        while True:
            # 가능한 행동을 조사한 후 표시
            available_action = env.get_action()
            print("possible actions = {}".format(available_action))

            # 상태 번호 표시
            print("+----+----+----+----+")
            print("+  0 +  1 +  2 +  3 +")
            print("+----+----+----+----+")
            print("+  4 +  5 +  6 +  7 +")
            print("+----+----+----+----+")
            print("+  8 +  9 + 10 + 11 +")
            print("+----+----+----+----+")
            print("+ 12 + 13 + 14 + 15 +")
            print("+----+----+----+----+")
                        
            # 키보드로 가능한 행동을 입력 받음
            action = input("Select action(human) : ")
            action = int(action)
            
            # 입력받은 행동이 가능한 행동이면 반복문을 탈출
            if action in available_action:
                return action
            # 아니면 행동 입력을 반복
            else:
                print("You selected wrong action")
        return

In [20]:
class Agent:

    # 확률에 음수가 있을 경우 전부 양수가 되도록 보정
    def positive_normalize(self, policy):
        min_val = min(policy.values())
        if min_val < 0:
            for action in policy:
                policy[action] -= min_val

        total_sum = sum(policy.values())

        if total_sum > 0:
            for action in policy:
                policy[action] /= total_sum
        else:
            for action in policy:
                policy[action] = 1.0 / len(policy)

    # 정책 기반 행동 선택
    def select_action(self, state, policy, available_actions):
        # 현재 상태의 정책이 없으면 균등 확률로 초기화
        if state not in policy:
            policy[state] = {}
            probability = 1.0 / len(available_actions)
            for action in available_actions:
                policy[state][action] = probability

        # Gibbs 소프트맥스 함수로 선택될 확률을 조정
        exp_values = {}
        for action in available_actions:
            if action in policy[state]:
                value = policy[state][action]
            else:
                value = 0.0
            exp_values[action] = np.exp(value)

        # 확률 계산
        total_exp_value = sum(exp_values.values())
        probabilities = {}
        if total_exp_value > 0:
            for action in available_actions:
                probabilities[action] = exp_values[action] / total_exp_value
        else:
            probability = 1.0 / len(available_actions)
            for action in available_actions:
                probabilities[action] = probability

        # 행동 선택
        actions = list(available_actions)
        pr = []
        for action in actions:
            pr.append(probabilities[action])

        i = np.random.choice(range(len(actions)), p=pr)
        return actions[i]

In [21]:
class Actor_Critic:
    is_trained = False
    trained_policy = {}
    trained_optimal_policy = {}
    
    def __init__(self):
        self.name = "Actor Critic"
        self.agent = Agent()
        
        # 학습 여부 확인
        if Actor_Critic.is_trained:
            self.policy = Actor_Critic.trained_policy
            self.optimal_policy = Actor_Critic.trained_optimal_policy
            return
        
        # 액터-크리틱 알고리즘 초기화
        np.random.seed(0)
        
        # 환경 초기화
        self.env = Environment()
        
        # 학습 파라미터 설정
        self.gamma = 0.9 
        self.alpha = 0.2  
        self.beta = 0.15  
        
        self.V = {}      
        self.policy = {}  
        self.optimal_policy = {}  
        
        self.max_episode = 14000
        self.max_step = 16      
        
        # 각 에피소드에 대해 반복:
        for epi in tqdm(range(self.max_episode)):
            # 환경 초기화
            self.env = Environment()
            
            # 에피소드의 각 스텝에 대해 반복:
            for k in range(self.max_step):
                current_state = tuple(self.env.board_a)
                
                # 행동 선택
                action = self.select_action(self.env, 1)
                if action is None:
                    break
                    
                # 환경에 행동 적용
                self.env.board_a[action] = 1
                
                # 보상과 종료 여부 확인
                self.env.end_check(1)
                reward = self.env.reward
                done = self.env.done
                
                # 다음 상태 s'
                next_state = tuple(self.env.board_a)
                
                # 새로운 상태 가치 함수 초기화
                if current_state not in self.V:
                    self.V[current_state] = 0.0
                if next_state not in self.V:
                    self.V[next_state] = 0.0
                
                # 크리틱 학습: 시간차 에러 계산
                td_error = reward + self.gamma * self.V[next_state] - self.V[current_state]
                
                # 상태 가치 함수 업데이트
                self.V[current_state] += self.alpha * td_error
                
                # 액터 정책 초기화 (필요한 경우)
                if current_state not in self.policy:
                    self.policy[current_state] = {}
                
                # 액터 학습: 행동 가치 업데이트
                if action not in self.policy[current_state]:
                    self.policy[current_state][action] = 0.0
                    
                self.policy[current_state][action] += td_error * self.beta
                
                # 확률에 음수가 있을 경우 전부 양수가 되도록 보정
                self.agent.positive_normalize(self.policy[current_state])
                
                # s가 마지막 상태라면 종료
                if done:
                    break
            
            # 학습된 정책에서 최적 행동을 추출
            for state in self.policy:
                if self.policy[state]:
                    self.optimal_policy[state] = max(self.policy[state], key=self.policy[state].get)
        
        # 학습 완료 후 학습된 정책 저장
        Actor_Critic.trained_policy = self.policy
        Actor_Critic.trained_optimal_policy = self.optimal_policy
        Actor_Critic.is_trained = True
    
    def select_action(self, env, player):
        state = tuple(env.board_a)
        available_actions = env.get_action()
        if not available_actions:
            return None
        return self.agent.select_action(state, self.policy, available_actions)

In [22]:
np.random.seed(0)

#p1 = Human_player()
p1 = Actor_Critic()

p2 = Human_player()
#p2 = Actor_Critic()

# 지정된 게임 수를 자동으로 두게 할 것인지 한게임씩 두게 할 것인지 결정
# auto = True : 지정된 판수(games)를 자동으로 진행 
# auto = False : 한판씩 진행

auto = False

# auto 모드의 게임수
games = 100

print("pl player : {}".format(p1.name))
print("p2 player : {}".format(p2.name))

# 각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0

if auto: 
    # 자동 모드 실행
    for j in tqdm(range(games)):
        
        np.random.seed(j)
        env = Environment()
        
        for i in range(10000):
            # p1 과 p2가 번갈아 가면서 게임을 진행
            # p1(1) -> p2(-1) -> p1(1) -> p2(-1) ...
            reward, done = env.move(p1,p2,(-1)**i)
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    p1_score += 1
                elif reward == -1:
                    p2_score += 1
                else:
                    draw_score += 1
                break

else:                
    # 한 게임씩 진행하는 수동 모드
    np.random.seed(1)
    while True:
        
        env = Environment()
        env.print = False
        for i in range(10000):
            reward, done = env.move(p1,p2,(-1)**i)
            env.print_board()
            if done == True:
                if reward == 1:
                    print("winner is p1({})".format(p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("winner is p2({})".format(p2.name))
                    p2_score += 1
                else:
                    print("draw")
                    draw_score += 1
                break
        
        # 최종 결과 출력        
        print("final result")
        env.print_board()

        # 한게임 더?최종 결과 출력 
        answer = input("More Game? (y/n)")

        if answer == 'n':
            break           

print("p1({}) = {} p2({}) = {} draw = {}".format(p1.name, p1_score,p2.name, p2_score,draw_score))

  2%|█▍                                                                           | 270/14500 [00:01<01:27, 162.27it/s]


KeyboardInterrupt: 